## Step 27: 테일러 급수 미분

sin 함수의 미분을 테일러 급수를 이용해 계산해보자.

### 27.1 sin 함수 구현

$$
y = sin(x)
$$
$$
{\delta y \over \delta x} = cos(x)
$$

```python
# step27.py
import numpy as np
from dezero import Function


class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = gy * np.cos(x)
        return gx


def sin(x):
    return Sin()(x)
```

```python
from dezero import Variable

x = Variable(np.array(np.pi/4))
y = sin(x)
y.backward()

print(y.data)
print(x.grad)
```

\> 0.7071067811865475

\> 0.7071067811865476

### 27.2 테일러 급수 이론

테일러 급수: 어떤 함수를 다항식으로 근사하는 방법

점 $a$에서 $f(x)$의 테일러 급수 식은 다음과 같다.
$$
f(x) = f(a) + f'(a)(x-a) + {1\over 2!}f''(a)(x-a)^2 + {1\over 3!}f'''(a)(x-a)^3 + ...
$$

$a=0$일 때의 테일러 급수를 매클로린 전개(Maclaurin's series)라고 한다.
$$
f(x) = f(0) + f'(0)x + {1\over 2!}f''(0)x^2 + {1\over 3!}f'''(0)x^3 + ...
$$

sin(0) = 0, cos(0) = 1이므로 sin(x)에 매클로린 전개를 적용하면 다음과 같은 식을 도출할 수 있다.
$$
sin(x) = \Sigma^\infty_{i=0}(-1)^i{x^{2i+1}\over(2i+1)^2!}
$$

### 27.3 테일러 급수 구현

```python
import math

def my_sin(x, threshold=0.0001):
    y = 0
    for i in range(100_000):
        c = (-1) ** i / math.factorial(2 * i + 1)
        t = c * x ** (2 * i + 1)
        y = y + t

        if abs(t.data) < threshold:
            break
    return y
```

threshold가 작을수록 정밀도가 높아진다.

```python
x = Variable(np.array(np.pi/4))
y = my_sin(x)
y.backward()

print(y.data)
print(x.grad)
```

### 27.4 계산 그래프 시각화

```python
x = Variable(np.array(np.pi/4))
y = my_sin(x)
y.backward()

x.name = 'x'
y.name = 'y'
plot_dot_graph(y, verbose=False, to_file='./Kangbeen/steps/step27-1.png')
```

#### threshold=0.0001일때
![step27-1](https://user-images.githubusercontent.com/76294398/184317924-adc8c109-d93a-4628-ad61-8e78e550d5fd.png)


#### threshold=1e-150일때
![step27-2](https://user-images.githubusercontent.com/76294398/184318200-06087a0f-1d15-4563-9b91-c9bbfd3b4003.png)